<a href="https://colab.research.google.com/github/wander-asb/US_Accidents/blob/main/Us.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Leitura de arquivos

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [11]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [12]:
import findspark
findspark.init()

In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('Inicializando com Spark').getOrCreate()

In [19]:
spark

In [21]:
from google.colab                import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [22]:
! mkdir ~/.kaggle

In [23]:
! cp /content/gdrive/MyDrive/kaggle_api/kaggle.json ~/.kaggle/kaggle.json

In [24]:
! chmod 600 ~/.kaggle/kaggle.json

In [26]:
! kaggle datasets download -d sobhanmoosavi/us-accidents

 98% 641M/653M [00:13<00:00, 91.8MB/s]
100% 653M/653M [00:13<00:00, 50.2MB/s]


In [27]:
! unzip /content/us-accidents.zip

Archive:  /content/us-accidents.zip
  inflating: US_Accidents_March23.csv  


In [28]:
data_dir = '/content/US_Accidents_March23.csv'

In [30]:
us_accident =  spark.read.csv(data_dir)

In [34]:
us_accident.show()

+----+-------+--------+-------------------+-------------------+------------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+--------------------+
| _c0|    _c1|     _c2|                _c3|                _c4|               _c5|               _c6|    _c7|    _c8|         _c9|                _c10|                _c11|        _c12|      _c13| _c14|      _c15|   _c16|      _c17|        _c18|               _c19|          _c20|         _c21|       _c22|        _c23|          _c24|          _c25|           _c26|             _c27|             _c28|  